# Pandasのトレーニング♨

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インストール](#インストール)
  - [インポート](#インポート)
  - [warningを非表示](#warningを非表示)
  - [プロキシ設定](#プロキシ設定)
- [Series](#Series)
  - [生成](#生成)
  - [アクセス](#アクセス)
- [DataFrame](#DataFrame)
  - [生成](#生成_)
  - [アクセス](#アクセス_)
  - [編集](#編集)
  - [型変換](#型変換)
  - [集計操作](#集計操作)
- [CRISP-DM](#CRISP-DM)
  - [データの理解](#データの理解)
  - [データの準備](#データの準備)
  
## 参考
開発基盤部会 Wiki
- Pandas  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?Pandas

## 環境準備

### インストール

In [ ]:
!pip install numpy
!pip install pandas
!pip install openpyxl

### インポート

In [2]:
import io
import requests

import numpy as np
import pandas as pd

### warningを非表示

In [3]:
import warnings
warnings.filterwarnings('ignore')

### プロキシ設定

In [4]:
import os
proxies = {
    "http": os.getenv("HTTP_PROXY"),
    "https": os.getenv("HTTPS_PROXY")
}

## Series

### 生成

#### 配列から

In [5]:
pd.Series([3,7,10,13])

0     3
1     7
2    10
3    13
dtype: int64

#### NumPyから
NumPyの配列から

In [6]:
numpy_array = np.array([3,7,10,13])
numpy_array

array([ 3,  7, 10, 13])

In [7]:
pd.Series(numpy_array)

0     3
1     7
2    10
3    13
dtype: int64

#### コピー

##### 以下は参照渡し

In [8]:
ss=pd.Series([3,7,10,13])
ss2=ss
ss2[1]=100
print(ss)
print(ss2)

0      3
1    100
2     10
3     13
dtype: int64
0      3
1    100
2     10
3     13
dtype: int64


##### コピーする場合は以下

In [9]:
ss=pd.Series([3,7,10,13])
ss2=ss.copy()
ss2[1]=100
print(ss)
print(ss2)

0     3
1     7
2    10
3    13
dtype: int64
0      3
1    100
2     10
3     13
dtype: int64


### アクセス

In [10]:
x=pd.Series([3,7,10,13])
x

0     3
1     7
2    10
3    13
dtype: int64

#### インデックス

In [11]:
x[1]

np.int64(7)

#### 抽出

##### スライシング

In [12]:
x[1:3]

1     7
2    10
dtype: int64

##### 配列でインデックスを指定
以下ではnp.arrayを使っているが、普通の配列でも指定可能。

In [13]:
x[np.array([1,2])]

1     7
2    10
dtype: int64

##### 比較演算

###### 5より大きい値を調査。

In [14]:
# npと似たような動作で、boolのseriesで返る
x>=5

0    False
1     True
2     True
3     True
dtype: bool

###### 5より大きい値を抽出。

In [15]:
# 上記のseriesでインデックスを指定する。
x[x>=5]

1     7
2    10
3    13
dtype: int64

#### 配列
NumPy配列の取り出し

In [16]:
x.values

array([ 3,  7, 10, 13])

## DataFrame

### 生成_

#### 辞書型から
列名と列配列から

In [17]:
df=pd.DataFrame({
    'xxx': ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'], # 長さ: 6
    'yyy': [20, 34, 50, 12, 62, 22],                   # 長さ: 6
    'zzz': ['AAA', 'BBB', 'CCC', 'DDD', 'EEE', 'FFF']  # 長さ: 6
})
df

,xxx,yyy,zzz
0,aaa,20,AAA
1,bbb,34,BBB
2,ccc,50,CCC
3,ddd,12,DDD
4,eee,62,EEE
5,fff,22,FFF


#### NumPyから_
NumPyの行列（２次配列）から

In [18]:
np_arr=np.array([
    ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'],  # 長さ: 6
    [20, 34, 50, 12, 62, 22],                    # 長さ: 6
    ['AAA', 'BBB', 'CCC', 'DDD', 'EEE', 'FFF']]) # 長さ: 6
df=pd.DataFrame(np_arr.T,columns=['xxx', 'yyy', 'zzz']) # 転置させるポイント
df

,xxx,yyy,zzz
0,aaa,20,AAA
1,bbb,34,BBB
2,ccc,50,CCC
3,ddd,12,DDD
4,eee,62,EEE
5,fff,22,FFF


In [19]:
# ファイル保存
df.to_csv('temp.csv')

#### CSVから

##### 事前にCSVファイルを配置しておく。

In [20]:
# df = pd.read_csv('temp.csv')
# df

##### CSVファイルをダウンロードして...
必要に応じて[環境変数にプロキシ設定](#プロキシ設定)をしておくと良い。

In [21]:
url = 'https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?plugin=attach&pcmd=open&file=temp.csv&refer=Pandas'
res = requests.get(url) # , verify=False, proxies=proxies) # プロキシ

###### 直接、読み込む。

In [22]:
df = pd.read_csv(io.BytesIO(res.content), encoding='shift-jis', sep=",")
df

,Unnamed: 0,xxx,yyy,zzz
0,0,aaa,20,AAA
1,1,bbb,34,BBB
2,2,ccc,50,CCC
3,3,ddd,12,DDD
4,4,eee,62,EEE
5,5,fff,22,FFF


###### 一旦保存して、読み込む。

In [23]:
with open('temp.csv', 'wb') as saveFile : saveFile.write(res.content)
df = pd.read_csv('temp.csv', encoding='shift-jis', sep=",")
df

,Unnamed: 0,xxx,yyy,zzz
0,0,aaa,20,AAA
1,1,bbb,34,BBB
2,2,ccc,50,CCC
3,3,ddd,12,DDD
4,4,eee,62,EEE
5,5,fff,22,FFF


#### Excelから
事前にopenpyxlのインストールが必要。

##### 事前にExcelファイルを配置しておく。

In [24]:
# df = pd.read_excel('temp.xlsx', engine='openpyxl')
# df

##### Excelファイルをダウンロードして...
必要に応じて[環境変数にプロキシ設定](#プロキシ設定)をしておくと良い。

In [25]:
url = 'https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?plugin=attach&pcmd=open&file=temp.xlsx&refer=Pandas'
res = requests.get(url) # , verify=False, proxies=proxies) # プロキシ

###### 直接、読み込む。

In [26]:
df = pd.read_excel(io.BytesIO(res.content), engine='openpyxl')
df

,Unnamed: 0,xxx,yyy,zzz
0,0,aaa,20,AAA
1,1,bbb,34,BBB
2,2,ccc,50,CCC
3,3,ddd,12,DDD
4,4,eee,62,EEE
5,5,fff,22,FFF


###### 一旦保存して、読み込む。

In [27]:
with open('temp.xlsx', 'wb') as saveFile : saveFile.write(res.content)
df = pd.read_excel('temp.xlsx', engine='openpyxl')
df

,Unnamed: 0,xxx,yyy,zzz
0,0,aaa,20,AAA
1,1,bbb,34,BBB
2,2,ccc,50,CCC
3,3,ddd,12,DDD
4,4,eee,62,EEE
5,5,fff,22,FFF


#### コピー

##### 以下は参照渡し

In [28]:
df=pd.DataFrame({
    'xxx': ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'], # 長さ: 6
    'yyy': [20, 34, 50, 12, 62, 22],                   # 長さ: 6
    'zzz': ['AAA', 'BBB', 'CCC', 'DDD', 'EEE', 'FFF']  # 長さ: 6
})
df2=df
df2.iloc[1,1]=100
print(df)
print(df2)

   xxx  yyy  zzz
0  aaa   20  AAA
1  bbb  100  BBB
2  ccc   50  CCC
3  ddd   12  DDD
4  eee   62  EEE
5  fff   22  FFF
   xxx  yyy  zzz
0  aaa   20  AAA
1  bbb  100  BBB
2  ccc   50  CCC
3  ddd   12  DDD
4  eee   62  EEE
5  fff   22  FFF


##### コピーする場合は以下

In [29]:
df=pd.DataFrame({
    'xxx': ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'], # 長さ: 6
    'yyy': [20, 34, 50, 12, 62, 22],                   # 長さ: 6
    'zzz': ['AAA', 'BBB', 'CCC', 'DDD', 'EEE', 'FFF']  # 長さ: 6
})
df2=df.copy()
df2.iloc[1,1]=100
print(df)
print(df2)

   xxx  yyy  zzz
0  aaa   20  AAA
1  bbb   34  BBB
2  ccc   50  CCC
3  ddd   12  DDD
4  eee   62  EEE
5  fff   22  FFF
   xxx  yyy  zzz
0  aaa   20  AAA
1  bbb  100  BBB
2  ccc   50  CCC
3  ddd   12  DDD
4  eee   62  EEE
5  fff   22  FFF


### アクセス_

#### 先頭・後尾

##### 先頭

###### 先頭５行を出力（既定値

In [30]:
df.head()

,xxx,yyy,zzz
0,aaa,20,AAA
1,bbb,34,BBB
2,ccc,50,CCC
3,ddd,12,DDD
4,eee,62,EEE


###### 先頭２行を出力（明示

In [31]:
df.head(2)

,xxx,yyy,zzz
0,aaa,20,AAA
1,bbb,34,BBB


##### 後尾

###### 後尾５行を出力（既定値

In [32]:
df.tail()

,xxx,yyy,zzz
1,bbb,34,BBB
2,ccc,50,CCC
3,ddd,12,DDD
4,eee,62,EEE
5,fff,22,FFF


###### 先頭２行を出力（明示

In [33]:
df.tail(2)

,xxx,yyy,zzz
4,eee,62,EEE
5,fff,22,FFF


#### インデックス

##### 行指定
Seriesが返る。

In [34]:
df.iloc[1]

xxx    bbb
yyy     34
zzz    BBB
Name: 1, dtype: object

##### 列指定
Seriesが返る。

In [35]:
# NumPyと同様にスライシングが可能
df.iloc[:,1]

0    20
1    34
2    50
3    12
4    62
5    22
Name: yyy, dtype: int64

##### 行列指定

###### 行列番号

In [36]:
df.iloc[1,0]

'bbb'

###### 行番号・列名
locは行名、列名が使える。

In [37]:
df.loc[1,'xxx']

'bbb'

#### 抽出

##### スライシング

###### 行

In [38]:
df.iloc[2:4]

,xxx,yyy,zzz
2,ccc,50,CCC
3,ddd,12,DDD


###### 列

In [39]:
df.iloc[:,2:4]

,zzz
0,AAA
1,BBB
2,CCC
3,DDD
4,EEE
5,FFF


##### 配列でインデックスを指定
以下ではnp.arrayを使っているが、普通の配列でも指定可能。

###### 行インデックス

In [40]:
df.iloc[np.array([2,3])]

,xxx,yyy,zzz
2,ccc,50,CCC
3,ddd,12,DDD


###### 行＆列インデックス

In [41]:
df.iloc[np.array([2,3]), np.array([1,2])]

,yyy,zzz
2,50,CCC
3,12,DDD


##### 比較演算
比較の結果boolのSeriesが生成されて指定されている。

###### 30より大きいyyy列の値を調査。

In [42]:
df.yyy >= 30

0    False
1     True
2     True
3    False
4     True
5    False
Name: yyy, dtype: bool

###### 30より大きいyyy列の値を抽出。

In [43]:
df[df.yyy >= 30]

,xxx,yyy,zzz
1,bbb,34,BBB
2,ccc,50,CCC
4,eee,62,EEE


#### 行列
NumPy行列の取り出し

In [44]:
df.values

array([['aaa', 20, 'AAA'],
       ['bbb', 34, 'BBB'],
       ['ccc', 50, 'CCC'],
       ['ddd', 12, 'DDD'],
       ['eee', 62, 'EEE'],
       ['fff', 22, 'FFF']], dtype=object)

#### 行指定（≒ 選択）

##### １行指定する場合はSeries

In [45]:
df.iloc[2]

xxx    ccc
yyy     50
zzz    CCC
Name: 2, dtype: object

##### 複数行指定（行を配列で指定）する場合はDataFrame

In [46]:
df.iloc[[2]]

,xxx,yyy,zzz
2,ccc,50,CCC


#### 列指定（≒ 射影）

##### １列指定する場合はSeries
行はスライシングで全行指定

In [47]:
df.loc[:,'yyy']

0    20
1    34
2    50
3    12
4    62
5    22
Name: yyy, dtype: int64

##### 複数列指定（列を配列で指定）する場合はDataFrame

###### 行列指定
行はスライシングで全行指定

In [48]:
df.loc[:,['yyy']]

,yyy
0,20
1,34
2,50
3,12
4,62
5,22


###### 列名のみ指定しても結果は同じ

In [49]:
df[['yyy']]

,yyy
0,20
1,34
2,50
3,12
4,62
5,22


### 編集
- 行編集: axis=0
- 列編集: axis=1

In [50]:
df=pd.DataFrame({
    'xxx': ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'], # 長さ: 6
    'yyy': [20, 34, 50, 12, 62, 22],                   # 長さ: 6
    'zzz': ['AAA', 'BBB', 'CCC', 'DDD', 'EEE', 'FFF']  # 長さ: 6
})
df

,xxx,yyy,zzz
0,aaa,20,AAA
1,bbb,34,BBB
2,ccc,50,CCC
3,ddd,12,DDD
4,eee,62,EEE
5,fff,22,FFF


#### 追加

##### 行追加

###### concatを使った例

In [51]:
row=pd.DataFrame({
           'xxx': 'ggg',
           'yyy': [45],
           'zzz': 'GGG'})

pd.concat([df,row],axis=0)

,xxx,yyy,zzz
0,aaa,20,AAA
1,bbb,34,BBB
2,ccc,50,CCC
3,ddd,12,DDD
4,eee,62,EEE
5,fff,22,FFF
0,ggg,45,GGG


###### インデックス再設定

In [52]:
df.index=np.arange(len(df))
df

,xxx,yyy,zzz
0,aaa,20,AAA
1,bbb,34,BBB
2,ccc,50,CCC
3,ddd,12,DDD
4,eee,62,EEE
5,fff,22,FFF


##### 列追加

###### 簡単な例

In [53]:
df['XXX']=['X1','X2','X3','X4','X5','X6']
df

,xxx,yyy,zzz,XXX
0,aaa,20,AAA,X1
1,bbb,34,BBB,X2
2,ccc,50,CCC,X3
3,ddd,12,DDD,X4
4,eee,62,EEE,X5
5,fff,22,FFF,X6


###### concatを使った例

In [54]:
col=pd.DataFrame({'YYY':['Y1','Y2','Y3','Y4','Y5','Y6']})
df=pd.concat([df,col],axis=1)
df

,xxx,yyy,zzz,XXX,YYY
0,aaa,20,AAA,X1,Y1
1,bbb,34,BBB,X2,Y2
2,ccc,50,CCC,X3,Y3
3,ddd,12,DDD,X4,Y4
4,eee,62,EEE,X5,Y5
5,fff,22,FFF,X6,Y6


#### 削除

##### 行削除

In [55]:
df=df.drop(2,axis=0)
df

,xxx,yyy,zzz,XXX,YYY
0,aaa,20,AAA,X1,Y1
1,bbb,34,BBB,X2,Y2
3,ddd,12,DDD,X4,Y4
4,eee,62,EEE,X5,Y5
5,fff,22,FFF,X6,Y6


##### 列削除

In [56]:
df=df.drop('yyy',axis=1)
df

,xxx,zzz,XXX,YYY
0,aaa,AAA,X1,Y1
1,bbb,BBB,X2,Y2
3,ddd,DDD,X4,Y4
4,eee,EEE,X5,Y5
5,fff,FFF,X6,Y6


#### 列名・行名の変更

##### 列名の変更

In [57]:
df.columns=['AAA','BBB','CCC','DDD']
df

,AAA,BBB,CCC,DDD
0,aaa,AAA,X1,Y1
1,bbb,BBB,X2,Y2
3,ddd,DDD,X4,Y4
4,eee,EEE,X5,Y5
5,fff,FFF,X6,Y6


##### 列名・行名の変更

In [58]:
df=df.rename(columns={'DDD': 'XXX'}, index={5: 'hoge'})
df

,AAA,BBB,CCC,XXX
0,aaa,AAA,X1,Y1
1,bbb,BBB,X2,Y2
3,ddd,DDD,X4,Y4
4,eee,EEE,X5,Y5
hoge,fff,FFF,X6,Y6


#### 列値の一括変更
列値を％に単位変更する（100倍する）などの一括変更。

In [59]:
df=pd.DataFrame({
     'xxx': ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'], # 長さ: 6
     'yyy': [20, 34, 50, 12, 62, 22], # 長さ: 6
     'zzz':['AAA', 'BBB', 'CCC', 'DDD', 'EEE', 'FFF'] # 長さ: 6
})
df

,xxx,yyy,zzz
0,aaa,20,AAA
1,bbb,34,BBB
2,ccc,50,CCC
3,ddd,12,DDD
4,eee,62,EEE
5,fff,22,FFF


In [60]:
df.iloc[:,1:2] = df.iloc[:,1:2] * 100
df

,xxx,yyy,zzz
0,aaa,2000,AAA
1,bbb,3400,BBB
2,ccc,5000,CCC
3,ddd,1200,DDD
4,eee,6200,EEE
5,fff,2200,FFF


#### 行列の入替（転置）

In [61]:
df.transpose()

,0,1,2,3,4,5
xxx,aaa,bbb,ccc,ddd,eee,fff
yyy,2000,3400,5000,1200,6200,2200
zzz,AAA,BBB,CCC,DDD,EEE,FFF


#### inplace=True
inplace=Trueを設定すると、元を変更する。  
※ drop、rename、dropnaなどメソッドに指定できる。

In [62]:
df.drop('yyy',axis=1)
df # 元のデータに変更は反映されない。

,xxx,yyy,zzz
0,aaa,2000,AAA
1,bbb,3400,BBB
2,ccc,5000,CCC
3,ddd,1200,DDD
4,eee,6200,EEE
5,fff,2200,FFF


In [63]:
df.drop('yyy',axis=1,inplace=True)
df # 元のデータに変更が反映される。

,xxx,zzz
0,aaa,AAA
1,bbb,BBB
2,ccc,CCC
3,ddd,DDD
4,eee,EEE
5,fff,FFF


### 型変換

#### SS ⇔ DF型変換

##### DF → SS
DFのアクセスの所で幾らか説明済み。

##### SS → DF 

In [64]:
df = pd.DataFrame(pd.Series([3,7,10,13]))
df

,0
0,3
1,7
2,10
3,13


#### NP ⇔ DF型変換

##### NP → DF
[NumPyから](#NumPyから_)

##### DF → NP 

###### np.arrayメソッド

In [65]:
np_arr=np.array(df)
np_arr

array([[ 3],
       [ 7],
       [10],
       [13]])

###### df.valuesプロパティ

In [66]:
np_arr=df.values
np_arr

array([[ 3],
       [ 7],
       [10],
       [13]])

### 集計操作

#### 行方向に集計 

In [67]:
df=pd.DataFrame({
    '0': ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09'], # 長さ: 10
    '1': ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19'], # 長さ: 10
    '2': ['20', '21', '22', '23', '24', '25', '26', '27', '28', '29'] # 長さ: 10
})
df=df.astype('int')
df

,0,1,2
0,0,10,20
1,1,11,21
2,2,12,22
3,3,13,23
4,4,14,24
5,5,15,25
6,6,16,26
7,7,17,27
8,8,18,28
9,9,19,29


##### 1～2列を行集計（スライシングでもOK

In [68]:
# 集計
ss_sum = df.iloc[:,[1,2]].sum(axis=0)
# 型変換
df_sum = pd.DataFrame(ss_sum)
# 列名変更
df_sum = df_sum.rename(columns={0: 'sum'})
df_sum

,sum
1,145
2,245


##### 集計結果を結合

In [69]:
df=pd.concat([df,df_sum.transpose()],axis=0)
df

,0,1,2
0,0.0,10,20
1,1.0,11,21
2,2.0,12,22
3,3.0,13,23
4,4.0,14,24
5,5.0,15,25
6,6.0,16,26
7,7.0,17,27
8,8.0,18,28
9,9.0,19,29


#### 列方向に集系 

In [70]:
df=pd.DataFrame({
    '0': ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09'], # 長さ: 10
    '1': ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19'], # 長さ: 10
    '2': ['20', '21', '22', '23', '24', '25', '26', '27', '28', '29'] # 長さ: 10
})
df=df.astype('int')
df

,0,1,2
0,0,10,20
1,1,11,21
2,2,12,22
3,3,13,23
4,4,14,24
5,5,15,25
6,6,16,26
7,7,17,27
8,8,18,28
9,9,19,29


##### 1～2列を列集計（スライシングでもOK

In [71]:
# 集計
ss_sum = df.iloc[:,[1,2]].sum(axis=1)
# 型変換
df_sum = pd.DataFrame(ss_sum)
# 列名変更
df_sum = df_sum.rename(columns={0: 'sum'})
df_sum

,sum
0,30
1,32
2,34
3,36
4,38
5,40
6,42
7,44
8,46
9,48


##### 集計結果を結合

In [72]:
df=pd.concat([df,df_sum],axis=1)
df

,0,1,2,sum
0,0,10,20,30
1,1,11,21,32
2,2,12,22,34
3,3,13,23,36
4,4,14,24,38
5,5,15,25,40
6,6,16,26,42
7,7,17,27,44
8,8,18,28,46
9,9,19,29,48


## CRISP-DM
DataFrameを用いた

- データの理解
- データの準備

あたりの処理。

### データの理解

In [73]:
df

,0,1,2,sum
0,0,10,20,30
1,1,11,21,32
2,2,12,22,34
3,3,13,23,36
4,4,14,24,38
5,5,15,25,40
6,6,16,26,42
7,7,17,27,44
8,8,18,28,46
9,9,19,29,48


#### 基本統計量

In [74]:
df.describe()

,0,1,2,sum
count,10.00000,10.00000,10.00000,10.000000
mean,4.50000,14.50000,24.50000,39.000000
std,3.02765,3.02765,3.02765,6.055301
min,0.00000,10.00000,20.00000,30.000000
25%,2.25000,12.25000,22.25000,34.500000
50%,4.50000,14.50000,24.50000,39.000000
75%,6.75000,16.75000,26.75000,43.500000
max,9.00000,19.00000,29.00000,48.000000


In [75]:
# 小数点以下、切捨
df.describe().astype(int)

,0,1,2,sum
count,10,10,10,10
mean,4,14,24,39
std,3,3,3,6
min,0,10,20,30
25%,2,12,22,34
50%,4,14,24,39
75%,6,16,26,43
max,9,19,29,48


#### 相関係数（相関行列）

In [76]:
df.corr()

,0,1,2,sum
0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0
sum,1.0,1.0,1.0,1.0


### データの準備

#### データのクリーニング

##### 補完処理 

In [77]:
def function1():
    NAN = np.nan
    df=pd.DataFrame({
        '0': ['00', '01', NAN, '03', '04', '05', '06', '07', '08', NAN], # 長さ: 10
        '1': ['10', '11', '12', '13', NAN, '15', NAN, '17', NAN, '19'], # 長さ: 10
        '2': [NAN, '21', '22', '23', '24', '25', NAN, '27', '28', '29'] # 長さ: 10
    })
    return df.astype(float)
df=function1()
df

,0,1,2
0,0.0,10.0,NaN
1,1.0,11.0,21.0
2,NaN,12.0,22.0
3,3.0,13.0,23.0
4,4.0,NaN,24.0
5,5.0,15.0,25.0
6,6.0,NaN,NaN
7,7.0,17.0,27.0
8,8.0,NaN,28.0
9,NaN,19.0,29.0


###### リストワイズ法（削除）
欠損値を含む行（列）を削除。
- how='any'が既定値で、how='all'だと全列（行）欠損値の場合に限定。
- axis=0の行を削除が既定値で、axis=1だと列を削除。

In [78]:
df = function1()
# 欠損値を含む行を削除
df.dropna()

,0,1,2
1,1.0,11.0,21.0
3,3.0,13.0,23.0
5,5.0,15.0,25.0
7,7.0,17.0,27.0


In [79]:
df = function1()
# 欠損していない列数が指定した値以上の行を削除
df.dropna(thresh=2)

,0,1,2
0,0.0,10.0,NaN
1,1.0,11.0,21.0
2,NaN,12.0,22.0
3,3.0,13.0,23.0
4,4.0,NaN,24.0
5,5.0,15.0,25.0
7,7.0,17.0,27.0
8,8.0,NaN,28.0
9,NaN,19.0,29.0


###### ペアワイズ法（削除）
指定の列（行）に欠損値を含む行（列）を削除
- how='any'が既定値で、how='all'だと全列（行）欠損値の場合に限定。
- axis=0の行を削除が既定値で、axis=1だと列を削除。

In [80]:
df = function1()
# 指定の列に欠損値を含む行を削除
df.dropna(subset=['1', '2'])

,0,1,2
1,1.0,11.0,21.0
2,NaN,12.0,22.0
3,3.0,13.0,23.0
5,5.0,15.0,25.0
7,7.0,17.0,27.0
9,NaN,19.0,29.0


###### 補完値で置換

In [81]:
df = function1()
# 平均値補完
df.fillna(df.mean())

,0,1,2
0,0.00,10.000000,24.875
1,1.00,11.000000,21.000
2,4.25,12.000000,22.000
3,3.00,13.000000,23.000
4,4.00,13.857143,24.000
5,5.00,15.000000,25.000
6,6.00,13.857143,24.875
7,7.00,17.000000,27.000
8,8.00,13.857143,28.000
9,4.25,19.000000,29.000


In [82]:
df = function1()
# 中央値補完
df.fillna(df.median())

,0,1,2
0,0.0,10.0,24.5
1,1.0,11.0,21.0
2,4.5,12.0,22.0
3,3.0,13.0,23.0
4,4.0,13.0,24.0
5,5.0,15.0,25.0
6,6.0,13.0,24.5
7,7.0,17.0,27.0
8,8.0,13.0,28.0
9,4.5,19.0,29.0


In [83]:
df = function1()
# 最頻値補完
df.fillna(df.mode().iloc[0])

,0,1,2
0,0.0,10.0,21.0
1,1.0,11.0,21.0
2,0.0,12.0,22.0
3,3.0,13.0,23.0
4,4.0,10.0,24.0
5,5.0,15.0,25.0
6,6.0,10.0,21.0
7,7.0,17.0,27.0
8,8.0,10.0,28.0
9,0.0,19.0,29.0


In [84]:
df=function1()
# 線形補完
df.interpolate(method='linear')

,0,1,2
0,0.0,10.0,NaN
1,1.0,11.0,21.0
2,2.0,12.0,22.0
3,3.0,13.0,23.0
4,4.0,14.0,24.0
5,5.0,15.0,25.0
6,6.0,16.0,26.0
7,7.0,17.0,27.0
8,8.0,18.0,28.0
9,8.0,19.0,29.0


###### 任意の値で補完

In [85]:
df=function1()
# 全列
df.fillna(10)

,0,1,2
0,0.0,10.0,10.0
1,1.0,11.0,21.0
2,10.0,12.0,22.0
3,3.0,13.0,23.0
4,4.0,10.0,24.0
5,5.0,15.0,25.0
6,6.0,10.0,10.0
7,7.0,17.0,27.0
8,8.0,10.0,28.0
9,10.0,19.0,29.0


In [86]:
df=function1()
# 列指定
df.fillna({'0':10, '1':20, '2':30})

,0,1,2
0,0.0,10.0,30.0
1,1.0,11.0,21.0
2,10.0,12.0,22.0
3,3.0,13.0,23.0
4,4.0,20.0,24.0
5,5.0,15.0,25.0
6,6.0,20.0,30.0
7,7.0,17.0,27.0
8,8.0,20.0,28.0
9,10.0,19.0,29.0


###### 前後の値で補完
更に、axis、limit等も指定可能。

In [87]:
df=function1()
# 前の値
df.fillna(method='ffill')

,0,1,2
0,0.0,10.0,NaN
1,1.0,11.0,21.0
2,1.0,12.0,22.0
3,3.0,13.0,23.0
4,4.0,13.0,24.0
5,5.0,15.0,25.0
6,6.0,15.0,25.0
7,7.0,17.0,27.0
8,8.0,17.0,28.0
9,8.0,19.0,29.0


In [88]:
df=function1()
# 後の値
df.fillna(method='bfill')

,0,1,2
0,0.0,10.0,21.0
1,1.0,11.0,21.0
2,3.0,12.0,22.0
3,3.0,13.0,23.0
4,4.0,15.0,24.0
5,5.0,15.0,25.0
6,6.0,17.0,27.0
7,7.0,17.0,27.0
8,8.0,19.0,28.0
9,NaN,19.0,29.0


###### Xを含む要素をXで一括置換

In [89]:
NAN = np.nan
df=pd.DataFrame({
    'XXX': ['abcAAAxyz', 'abcBBBxyz'],
    'YYY': ['abcBBBxyz', 'abcAAAxyz']
})
df

,XXX,YYY
0,abcAAAxyz,abcBBBxyz
1,abcBBBxyz,abcAAAxyz


In [90]:
df['XXX'] = df['XXX'].str.replace('.*(AAA).*', 'CCC', regex=True)
df

,XXX,YYY
0,CCC,abcBBBxyz
1,abcBBBxyz,abcAAAxyz


###### 要素を要素中の[,]と[.]で囲まれた文字列で一括置換

In [91]:
NAN = np.nan
df=pd.DataFrame({
    'XXX': ['AAA,BBB.CCC', 'XXX,YYY.ZZZ'],
    'YYY': ['AAA,BBB.CCC', 'XXX,YYY.ZZZ']
})
df

,XXX,YYY
0,"AAA,BBB.CCC","AAA,BBB.CCC"
1,"XXX,YYY.ZZZ","XXX,YYY.ZZZ"


In [92]:
list_col1 = [i.split(",")[1].split(".")[0].strip() for i in df["XXX"]]
df["XXX"] = pd.Series(list_col1)
nparr_col1 = df["XXX"].unique()
for cell in nparr_col1:
   df['XXX'] = df['XXX'].str.replace('.*({0}\.).*'.format(cell), cell, regex=True)
df

,XXX,YYY
0,BBB,"AAA,BBB.CCC"
1,YYY,"XXX,YYY.ZZZ"


###### グループの平均値で補完

In [93]:
NAN = np.nan
df=pd.DataFrame({
    'XXX': [1, 2, 3, NAN, 5, 6, 7, 8, 9, 10],                 # 長さ: 10
    'YYY': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A'] # 長さ: 10
})
df

,XXX,YYY
0,1.0,A
1,2.0,B
2,3.0,C
3,NaN,A
4,5.0,B
5,6.0,C
6,7.0,A
7,8.0,B
8,9.0,C
9,10.0,A


In [94]:
# df['補完対象列名'] = df.groupby('グループ化列名')['補完対象列名'].transform...
df['XXX'] = df.groupby('YYY')['XXX'].transform(lambda d: d.fillna(d.mean()))
df # XXX列3番行が (1+7+10) / 3 = 6 で補完される。

,XXX,YYY
0,1.0,A
1,2.0,B
2,3.0,C
3,6.0,A
4,5.0,B
5,6.0,C
6,7.0,A
7,8.0,B
8,9.0,C
9,10.0,A


#### データの構築

##### 単一属性変換

###### カテゴリ → 数値

In [95]:
# df['列名'] = df['列名'].map({'カテゴリ1':数値1, 'カテゴリ2':数値2, ..., 'カテゴリn':数値n})
df2 = df.copy()
df['YYY'] = df['YYY'].map({'A':1, 'B':2, 'C':3})
df

,XXX,YYY
0,1.0,1
1,2.0,2
2,3.0,3
3,6.0,1
4,5.0,2
5,6.0,3
6,7.0,1
7,8.0,2
8,9.0,3
9,10.0,1


###### 数値 → カテゴリ

In [96]:
# df['列名'] = df['列名'].map({数値1:'カテゴリ1', 数値2:'カテゴリ2', ..., 数値n:'カテゴリn'})
df2['YYY'] = df['YYY'].map({1:'A', 2:'B', 3:'C'})
df2

,XXX,YYY
0,1.0,A
1,2.0,B
2,3.0,C
3,6.0,A
4,5.0,B
5,6.0,C
6,7.0,A
7,8.0,B
8,9.0,C
9,10.0,A


#### データの統合

##### One-Hotエンコーディング
- Pandasの場合、get_dummiesを使う。
- [NumPyの場合、to_categoricalを使う。](NumPyTraining.ipynb)

###### カテゴリを列に展開（0・1化）など。

In [97]:
pd.get_dummies(df['YYY'])

,1,2,3
0,True,False,False
1,False,True,False
2,False,False,True
3,True,False,False
4,False,True,False
5,False,False,True
6,True,False,False
7,False,True,False
8,False,False,True
9,True,False,False


###### 展開した列を表に追加する流れ

In [98]:
df_dummy = pd.get_dummies(df['YYY'])
df = pd.concat([df, df_dummy], axis=1)
df

,XXX,YYY,1,2,3
0,1.0,1,True,False,False
1,2.0,2,False,True,False
2,3.0,3,False,False,True
3,6.0,1,True,False,False
4,5.0,2,False,True,False
5,6.0,3,False,False,True
6,7.0,1,True,False,False
7,8.0,2,False,True,False
8,9.0,3,False,False,True
9,10.0,1,True,False,False
